In [1]:
!git clone https://github.com/adikan2k/Final-Project-Group-LexiCore.git
%cd Final-Project-Group-LexiCore

Cloning into 'Final-Project-Group-LexiCore'...
remote: Enumerating objects: 415, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 415 (delta 8), reused 1 (delta 1), pack-reused 384 (from 2)
Receiving objects: 100% (415/415), 319.87 MiB | 16.90 MiB/s, done.
Resolving deltas: 100% (166/166), done.
/content/Final-Project-Group-LexiCore


In [2]:
!git checkout trisha-singh

Updating files: 100% (55/55), done.
Branch 'trisha-singh' set up to track remote branch 'trisha-singh' from 'origin'.
Switched to a new branch 'trisha-singh'


In [3]:
!ls

'500 Samples'  'Final Group Presentation'     notebooks   requirements.txt
 code	       'Final Group Project Report'   Outputs	  simple_demo.py
 data	        LICENSE			      README.md   src


In [4]:
!ls data

embeddings  processed  raw


In [5]:
import pandas as pd

df = pd.read_parquet("data/processed/cleaned_papers.parquet")
print(df.shape)
df.head()

(500, 14)


,paper_id,title,authors,original_abstract,cleaned_text,language,sentences,num_sentences,tokens,processed_text,num_tokens,source,year,venue
0,arxiv_2511.15703v2,"Think Visually, Reason Textually: Vision-Langu...","[Beichen Zhang, Yuhang Zang, Xiaoyi Dong, Yuha...",Abstract reasoning from minimal examples remai...,abstract reasoning from minimal examples remai...,en,[Abstract reasoning from minimal examples rema...,1,"[abstract, reasoning, minimal, example, remain...",abstract reasoning minimal example remain core...,173,arxiv,2025,arXiv
1,arxiv_2511.16613v1,Rate-optimal community detection near the KS t...,"[Jingqiu Ding, Yiding Hua, Kasper Lindberg, Da...",We study community detection in the \emph{symm...,we study community detection in the emphsymmet...,en,[We study community detection in the emphsymme...,1,"[study, community, detection, emphsymmetric, s...",study community detection emphsymmetric stocha...,133,arxiv,2025,arXiv
2,arxiv_2511.20992v1,Dataset Poisoning Attacks on Behavioral Clonin...,"[Akansha Kalra, Soumil Datta, Ethan Gilmore, D...",Behavior Cloning (BC) is a popular framework f...,behavior cloning bc is a popular framework for...,en,[Behavior Cloning BC is a popular framework fo...,1,"[behavior, clone, popular, framework, train, s...",behavior clone popular framework train sequent...,127,arxiv,2025,arXiv
3,arxiv_2511.15120v1,Neural Networks Learn Generic Multi-Index Mode...,"[Bohan Zhang, Zihao Wang, Hengyu Fu, Jason D. ...","In deep learning, a central issue is to unders...","in deep learning, a central issue is to unders...",en,"[In deep learning, a central issue is to under...",1,"[deep, learning, central, issue, understand, n...",deep learning central issue understand neural ...,126,arxiv,2025,arXiv
4,s2orc_e31c6664182777a24b19ef09993ded07d1ed326e,Pyramid: A Layered Model for Nested Named Enti...,"[Jue Wang, Lidan Shou, Ke Chen, Gang Chen]","This paper presents Pyramid, a novel layered m...","this paper presents pyramid, a novel layered m...",en,"[This paper presents Pyramid, a novel layered ...",1,"[paper, present, pyramid, novel, layered, mode...",paper present pyramid novel layered model nest...,90,s2orc,2020,Annual Meeting of the Association for Computat...


In [6]:
!pip install sumy transformers sentence-transformers faiss-cpu rank-bm25 lime --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 108.6 MB/s eta 0:00:00


In [7]:
import os
print("CWD before:", os.getcwd())
%cd /content/Final-Project-Group-LexiCore
!git checkout trisha-singh
!git status

CWD before: /content/Final-Project-Group-LexiCore
/content/Final-Project-Group-LexiCore
Already on 'trisha-singh'
Your branch is up to date with 'origin/trisha-singh'.
On branch trisha-singh
Your branch is up to date with 'origin/trisha-singh'.

nothing to commit, working tree clean


In [8]:
%%writefile code/summarization.py
"""
 Summarization Engine

Provides:
- extractive_summary_textrank
- abstractive_summary_bart
- generate_summaries (1-sentence, 3-sentence, 5-bullet)
"""

from typing import Dict, List

from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer

from transformers import pipeline



def extractive_summary_textrank(text: str, num_sentences: int = 3) -> str:
    """
    Extractive summary using TextRank (Sumy).

    Args:
        text: input document (abstract)
        num_sentences: how many sentences to keep

    Returns:
        Summary as a single string.
    """
    if not isinstance(text, str) or len(text.strip()) == 0:
        return "No text available."

    try:
        parser = PlaintextParser.from_string(text, Tokenizer("english"))
        summarizer = TextRankSummarizer()
        summary_sents = summarizer(parser.document, num_sentences)
        sentences = [str(s) for s in summary_sents]
        if not sentences:
            return "Summary unavailable (text too short)."
        return " ".join(sentences)
    except Exception as e:
        return f"Extractive summary error: {e}"



_bart_summarizer = None

def _get_bart_pipeline():
    global _bart_summarizer
    if _bart_summarizer is None:
        _bart_summarizer = pipeline(
            "summarization",
            model="facebook/bart-large-cnn"
        )
    return _bart_summarizer


def abstractive_summary_bart(
    text: str,
    max_length: int = 120,
    min_length: int = 30,
) -> str:
    """
    Abstractive summary using BART.

    Args:
        text: input document
        max_length, min_length: summary length control

    Returns:
        Summary string.
    """
    if not isinstance(text, str) or len(text.strip()) == 0:
        return "No text available."

    try:
        summarizer = _get_bart_pipeline()
        truncated = text[:1024]   # BART input limit
        out = summarizer(
            truncated,
            max_length=max_length,
            min_length=min_length,
            do_sample=False,
        )
        return out[0]["summary_text"]
    except Exception as e:
        return f"Abstractive summary error: {e}"



def generate_summaries(text: str) -> Dict[str, object]:
    """
    Generates:
    - one_sentence: short abstractive summary
    - three_sentence: extractive summary (3 sentences)
    - five_bullets: list of bullet strings

    Returns:
        dict with keys: one_sentence, three_sentence, five_bullets
    """
    one_sentence = abstractive_summary_bart(
        text, max_length=40, min_length=15
    )
    three_sentence = extractive_summary_textrank(
        text, num_sentences=3
    )

    long_sum = abstractive_summary_bart(
        text, max_length=160, min_length=40
    )

    raw_bullets = [s.strip() for s in long_sum.split(".") if s.strip()]
    raw_bullets = raw_bullets[:5]
    bullets: List[str] = [f"• {b}" for b in raw_bullets]

    return {
        "one_sentence": one_sentence,
        "three_sentence": three_sentence,
        "five_bullets": bullets,
    }

Writing code/summarization.py
